<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-2/blob/main/CS6910_Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS6910-Assignment2-Part B
Using Pretrained model for Image Classification

In [2]:
#!pip install -q wandb
#import wandb
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [3]:
def get_data():
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

    # Training dataset:
    train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='training'
    )
    # Validation dataset:
    valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='validation'
    )
    
    #return train_data, valid_data
    return train_dataset, valid_dataset


In [25]:
def get_augmented_data():
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest",
                                        validation_split = 0.1)

    # Validation data is not being augmented
    validation_data_augmentation=ImageDataGenerator(
        validation_split=0.1
    )

    train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
    valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')

    return train_aug, valid_aug


#Transfer Learning 
1. Reference 1: https://www.tensorflow.org/tutorials/images/transfer_learning
2. Reference 2: https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751

In [29]:
class build_cnn(Model):
  
  def __init__(self,model_name,tune=False,freeze=20):
    
    super(build_cnn,self).__init__()
    
    self.image_shape=(256,256,3)
    self.model_name=model_name
    self.initialize_model()

    if tune == True:
      self.model.trainable= True
      train_and_tune=len(self.model.layers)-freeze

      for layer in self.model.layers[:freeze]:
        layer.trainable = False
    else:
      self.model.trainable = False

  # here we are using global average pooling layer as classifier on top of 
  # the pretrained convolution layers 
    self.avg_pool=layers.GlobalAveragePooling2D()
    self.dense=layers.Dense(10)
  
  # Initializing the model
  # NOTE: By using include_top=False, we are not using the final classification
  # layer of the pre trained model.
  def inceptionv3(self):
      model=tf.keras.applications.InceptionV3(input_shape=self.image_shape,
                                            include_top=False,
                                            weights='imagenet')
      return model

  def resnet50(self):
      model=tf.keras.applications.ResNet50(input_shape=self.image_shape,
                                         include_top=False,
                                         weights='imagenet')
      return model

  def inceptionresnetv2(self):
      model=tf.keras.applications.InceptionResNetV2(input_shape=self.image_shape,
                                                        include_top=False,
                                                        weights='imagenet')
      return model
  
  def xception(self):
      model=tf.keras.applications.Xception(input_shape=self.image_shape,
                                         include_top=False,
                                         weights='imagenet')
      return model
    
  
  def initialize_model(self):
    
    if self.model_name  == "InceptionV3":
      self.model=self.inceptionv3()
    
    elif self.model_name == "InceptionResNetV2":
      self.model=self.inceptionresnetv2()
    
    elif self.model_name == "ResNet50":
      self.model=self.resnet50()

    elif self.model_name == "Xception":
      self.model=self.xception()
    
    else:
      print("Enter a valid model name")
  
  
  def call(self,inputs):
    a=self.model(inputs)
    a=self.avg_pool(a)
    a=self.dense(a)
    return a

In [ ]:
base_model = "ResNet50"
tune = False
freeze = 20

model = build_cnn(base_model, tune, freeze)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

train_aug_gen=get_augmented_data()[0]
val_aug_gen=get_augmented_data()[1]

history = model.fit(train_aug_gen, steps_per_epoch=62, \
                    epochs=10, validation_data=val_aug_gen, \
                    validation_steps=62, workers=8)